# Live Black Scholes Pricing Model Using JAX

##### Zach Petko - 3/13/2025 

---

### Personal Background and Motivations for This Project
The motivation behind creating this Black-Scholes pricing model stems from my deep interest in the intersection of finance and technology. I’ve always been fascinated by how mathematical models can be applied to real-world financial markets to manage risk and make informed investment decisions. As a student eager to explore quantitative finance, I wanted to gain hands-on experience by implementing one of the most fundamental models in options pricing. The opportunity to work with live market data and apply advanced computational tools like JAX made this project especially appealing. By building this model, I aimed to enhance my understanding of option pricing theory, improve my programming skills, and explore how modern computing techniques can optimize financial calculations. This project not only allows me to apply theoretical knowledge to practical scenarios but also prepares me for a future career in quantitative analysis or financial engineering, where I hope to contribute to the evolution of financial modeling and risk management strategies. Thank you for viewing my work and I hope you enjoy!



---

## Table of Contents

### 1. Introduction
Explains the background of the Black-Scholes model and the motivations for working on this project.

### 2. Data Collection

Details the process of gathering real-time data including options, stock, and treasury information from Yahoo Finance.

### 3. Black-Scholes Model

Describes the Black-Scholes pricing formulas and their implementation. The model created for this project is compared to existing implementations in python from the BlackScholes library.

### 4. Implied Volatility Calculation Using Newton Method

Calculates an option's implied volatility (IV) using the Newton-Raphson method to aproximate the solution for IV.

---

## 1. Introduction

Accurately pricing options is a critical component of modern financial markets, enabling traders and investors to manage risk and make informed decisions. The **Black-Scholes model**, introduced in 1973 by Fischer Black, Myron Scholes, and Robert Merton, is one of the most widely used mathematical models for option pricing. It calculates the theoretical price of European-style options by considering factors such as the underlying asset price, strike price, time to expiration, volatility, risk-free interest rate, and dividends.

This project implements a live Black-Scholes pricing model using **JAX**, a high-performance library for automatic differentiation and numerical computing. The model is designed to price call options in real-time using market data from Yahoo Finance, and its performance is compared to an existing Black-Scholes implementation.

---

## 2. Data Collection

### Install Packages

In [133]:
#run in terminal to install packages: pip install package 

import yfinance as yf # yahoo finance 
import pandas as pd # pandas
import jax.numpy as jnp # jax numpy
from jax import grad # gradiant from jax
from jax.scipy.stats import norm as jnorm # norm density from jax

import blackscholes as bs # existing implementation to check work



### Load Options Data

Assign a valid stock ticker to ticker_input then run the code block

In [134]:
#input stock ticker
ticker_input = "AAPL" 


#create yf ticker for inputed ticker
ticker = yf.Ticker(ticker_input) 

#get all option expirations
expirations = ticker.options 

#create empty dataframes to add to with the loop
all_options = pd.DataFrame()
all_calls = pd.DataFrame()
all_puts = pd.DataFrame()

#loop through expirations and get option chain
for exp in expirations: 
    options_chain = ticker.option_chain(exp)
    calls = options_chain.calls
    puts = options_chain.puts
    calls['expiration'] = exp
    puts['expiration'] = exp
    calls['optionType'] = 'Call'
    puts['optionType'] = 'Put'

    #concatinate calls and puts for each expiration
    all_options = pd.concat([all_options, calls, puts], ignore_index=True)
    all_calls = pd.concat([all_calls, calls], ignore_index=True)
    all_puts = pd.concat([all_puts, puts], ignore_index=True)
#display options table
display(all_options)


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration,optionType
0,AAPL250321C00005000,2025-03-13 13:30:06+00:00,5.0,207.01,207.40,209.25,-3.410004,-1.62057,1.0,43,15.593750,True,REGULAR,USD,2025-03-21,Call
1,AAPL250321C00010000,2025-03-12 13:42:10+00:00,10.0,203.00,202.45,204.20,-8.460007,-4.00076,1.0,30,11.855471,True,REGULAR,USD,2025-03-21,Call
2,AAPL250321C00015000,2025-03-07 19:12:10+00:00,15.0,223.85,197.40,199.25,0.000000,0.00000,1.0,0,10.179691,True,REGULAR,USD,2025-03-21,Call
3,AAPL250321C00020000,2025-03-12 15:28:19+00:00,20.0,196.55,192.45,194.20,0.000000,0.00000,8.0,1,8.863286,True,REGULAR,USD,2025-03-21,Call
4,AAPL250321C00025000,2025-02-12 16:26:48+00:00,25.0,210.01,187.40,189.25,0.000000,0.00000,2.0,2,8.068364,True,REGULAR,USD,2025-03-21,Call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,AAPL270617P00410000,2025-02-14 14:39:31+00:00,410.0,166.68,194.00,199.00,0.000000,0.00000,3.0,0,0.267433,True,REGULAR,USD,2027-06-17,Put
1913,AAPL270617P00420000,2025-02-13 14:44:10+00:00,420.0,183.10,204.00,209.00,0.000000,0.00000,3.0,0,0.274665,True,REGULAR,USD,2027-06-17,Put
1914,AAPL270617P00430000,2025-02-12 14:45:18+00:00,430.0,198.33,214.00,219.00,0.000000,0.00000,3.0,0,0.281715,True,REGULAR,USD,2027-06-17,Put
1915,AAPL270617P00440000,2025-03-03 14:30:10+00:00,440.0,197.65,224.00,229.00,0.000000,0.00000,1.0,0,0.288520,True,REGULAR,USD,2027-06-17,Put


### Filter Call and Put Dataframes

In [135]:
#select columns
all = all_options[['bid', 'ask', 'strike', 'impliedVolatility', 'lastPrice', 'expiration', 'optionType']].copy()

#add a ticker column
all['ticker'] = ticker_input

#create time till expiration column
today_date = pd.to_datetime("today").normalize()
all['expiration'] = pd.to_datetime(all['expiration'])
all['yearsToMaturity'] = ((all['expiration'] - today_date).dt.days)/365 #time to expiration is in years

#filter out options with 0 values
all = all[all['lastPrice']>0]
all = all[all['strike']>0]
all = all[all['impliedVolatility']>0]
all = all[all['yearsToMaturity']>0.01] #this is to take out options maturing very soon so IV calculations break

#split to calls and puts datasets
calls = all[all['optionType']=='Call']
puts = all[all['optionType']=='Put']



display("Call Options: ", calls, "Put Options:", puts)


'Call Options: '

,bid,ask,strike,impliedVolatility,lastPrice,expiration,optionType,ticker,yearsToMaturity
0,207.40,209.25,5.0,15.593750,207.01,2025-03-21,Call,AAPL,0.019178
1,202.45,204.20,10.0,11.855471,203.00,2025-03-21,Call,AAPL,0.019178
2,197.40,199.25,15.0,10.179691,223.85,2025-03-21,Call,AAPL,0.019178
3,192.45,194.20,20.0,8.863286,196.55,2025-03-21,Call,AAPL,0.019178
4,187.40,189.25,25.0,8.068364,210.01,2025-03-21,Call,AAPL,0.019178
...,...,...,...,...,...,...,...,...,...
1859,2.26,3.15,410.0,0.278968,3.20,2027-06-17,Call,AAPL,2.260274
1860,2.10,2.77,420.0,0.278633,2.40,2027-06-17,Call,AAPL,2.260274
1861,1.69,2.48,430.0,0.279304,2.35,2027-06-17,Call,AAPL,2.260274
1862,1.68,2.24,440.0,0.280403,1.95,2027-06-17,Call,AAPL,2.260274


'Put Options:'

,bid,ask,strike,impliedVolatility,lastPrice,expiration,optionType,ticker,yearsToMaturity
83,0.0,0.01,5.0,8.250005,0.01,2025-03-21,Put,AAPL,0.019178
84,0.0,0.01,10.0,6.750002,0.01,2025-03-21,Put,AAPL,0.019178
85,0.0,0.03,15.0,6.312502,0.01,2025-03-21,Put,AAPL,0.019178
86,0.0,0.00,20.0,0.500005,0.01,2025-03-21,Put,AAPL,0.019178
87,0.0,0.00,25.0,0.500005,0.01,2025-03-21,Put,AAPL,0.019178
...,...,...,...,...,...,...,...,...,...
1912,194.0,199.00,410.0,0.267433,166.68,2027-06-17,Put,AAPL,2.260274
1913,204.0,209.00,420.0,0.274665,183.10,2027-06-17,Put,AAPL,2.260274
1914,214.0,219.00,430.0,0.281715,198.33,2027-06-17,Put,AAPL,2.260274
1915,224.0,229.00,440.0,0.288520,197.65,2027-06-17,Put,AAPL,2.260274


### Load Stock Price and Yield

In [136]:
#stock price
stockPrice = ticker.fast_info['lastPrice']

print(f"{ticker_input} current stock price is : ${stockPrice}")

#dividendYield
dividendYield = float(ticker.info.get('dividendYield') or 0)
print(f"{ticker_input} current Dividend Yield is : {dividendYield}")

AAPL current stock price is : $213.49000549316406
AAPL current Dividend Yield is : 0.48


### Load Risk Free Rate
Input appropriate treasury yield matching time to expiration

In [137]:
#^IRX: 3-month Treasury bill
#^FVX: 5-year Treasury note
#^TNX: 10-year Treasury note
#^TYX: 30-year Treasury bond

#import from yahoo finance
riskFreeRate = yf.Ticker("^TNX").history(period="1d")['Close'].iloc[-1] / 100

print(f"Risk Free Rate: {riskFreeRate}")


Risk Free Rate: 0.04308000087738037


---

## 3. Black-Scholes Model 

### Call Option Price:

$$
C = S e^{-q T} N(d_1) - K e^{-r T} N(d_2)
$$

### Put Option Price:

$$
P = K e^{-r T} N(-d_2) - S e^{-q T} N(-d_1)
$$

- S = Current stock price  
- K = Strike price  
- T = Time to maturity (in years)  
- r = Risk-free interest rate (annual)
- $\sigma$ = Volatility
- q = Dividend yield  
- N(x) = Standard Normal Cumulative distribution function (CDF)

$$
d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r - q + \frac{\sigma^2}{2}\right) T}{\sigma \sqrt{T}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T}
$$


### Black-Scholes Function

In [138]:
def black_scholes(S, K, T, r, sigma, q=0, optType='call'):
    d1 = (jnp.log(S / K) + (r - q + 0.5 * sigma **2 ) * T ) / (sigma * jnp.sqrt(T))
    d2 = d1 - sigma * jnp.sqrt(T)
    if optType == 'call':
        call = S * jnp.exp(-q * T) * jnorm.cdf(d1, 0, 1) - K * jnp.exp(-r * T) * jnorm.cdf(d2, 0, 1)
        return call
    else:
        put = K * jnp.exp(-r * T) * jnorm.cdf(-d2, 0, 1) - S * jnp.exp(-q * T) * jnorm.cdf(-d1, 0, 1)
        return put

In [139]:
# Our Model

#create empty list to collect values
estCallPrices = []  

S = stockPrice
r = riskFreeRate
q=dividendYield

for i in range(len(calls)):
    K = calls.iloc[i]['strike']
    T = calls.iloc[i]['yearsToMaturity']
    sigma = calls.iloc[i]['impliedVolatility']
    
    #calculate price using black scholes and append to estCallPrices
    price = black_scholes(S, K, T, r, sigma, q, optType='call')  
    estCallPrices.append(price)  

# Convert list to DataFrame at the end
estCallPrices = pd.Series(estCallPrices)



# Existing library

#create empty list to collect values
estCallPrices2 = []  

S = stockPrice
r = riskFreeRate
q=dividendYield

for i in range(len(calls)):
    K = calls.iloc[i]['strike']
    T = calls.iloc[i]['yearsToMaturity']
    sigma = calls.iloc[i]['impliedVolatility']
    
    #calculate price using black scholes and append to estCallPrices
    price = bs.BlackScholesCall(S=S, K=K, T=T, r=r, sigma=sigma, q=q).price() 
    estCallPrices2.append(price)  

# Convert list to DataFrame at the end
estCallPrices2 = pd.Series(estCallPrices2)




#Combine all in a df to compare
compEstCallPrices = pd.DataFrame({'Real Call Price' : calls['lastPrice'], 
                                 'Our Model Call Price' : estCallPrices,
                                  'Existing Model Call Price' : estCallPrices2})
display(compEstCallPrices.sample(n=10))




,Real Call Price,Our Model Call Price,Existing Model Call Price
1626,2.83,NaN,NaN
406,0.02,0.0065061226,0.006506
837,3.25,9.845015,9.845023
986,NaN,0.00038659712,0.000387
938,24.45,25.915438,25.915438
96,NaN,8.701035,8.701021
890,NaN,21.323112,21.323109
135,NaN,5.8465347,5.846546
1052,10.90,NaN,NaN
20,106.75,106.73938,106.739399


We see that our model performs almost exactly the same as the existing model. This makes sense since they should be the same model. Neither fits the Real Call Price very well, but this makes sense because we are using the black scholes model, one that works for European style options where the owner of the option can only choose to excercise on the maturity date. These prices are for American style options where the owner of the call option can choose to exercise on any date up until the maturity date. To correctly price American options, we need a different model.

### The Greeks
Use JAX to perform the partial differentiation to calculate the values. grad() computes the gradiant of the inputted function with respect to the variable at index n in the function def. After running grad it the assigned name now becomes a callable function.

#### **Delta - $\Delta$**
How much does the option price change if the underlying price changes?
$$
\Delta = \frac{\partial C}{\partial S}
$$
- C = Price of Call Option
- S = Current Stock Price

In [140]:
jax_delta = grad(black_scholes, argnums = 0)

#### **Gamma - $\Gamma$**
How much does the option price change if the volatility changes?
$$
\Gamma = \frac{\partial^2 C}{\partial S^2} = \frac{\partial \Delta}{\partial S}
$$
- C = Price of Call Option
- S = Current Stock Price

In [141]:
jax_gamma = grad(jax_delta, argnums = 0)

#### **Theta - $\Theta$**
How much does the option price change if the time to maturity changes?
$$
\Gamma = \frac{\partial C}{\partial T}
$$
- C = Price of Call Option
- T = Time to Maturity

In [142]:
jax_theta = grad(black_scholes, argnums = 2)

#### **Rho - $\rho$**
How much does the option price change if the intrest rate changes?
$$
\rho = \frac{\partial C}{\partial r}
$$
- C = Price of Call Option
- r = Risk Free Rate

In [143]:
jax_rho = grad(black_scholes, argnums = 3)

#### **Vega - $\nu$**
How much does the option price change if the volatility changes?
$$
\nu = \frac{\partial C}{\partial \sigma}
$$
- C = Price of Call Option
- \sigma = Volatility

In [144]:
jax_vega = grad(black_scholes, argnums = 4)

### Greeks for Options

In [145]:
#select number of options to calculate for, keep small
n = 10
#n= len(calls)




#Our model

#create empty list to collect values
callGreeks = pd.DataFrame(columns=['Delta', 'Gamma', 'Theta', 'Rho', 'Vega'])  

S = stockPrice
r = riskFreeRate
q=dividendYield

for i in range(n):
    K = calls.iloc[i]['strike']
    T = calls.iloc[i]['yearsToMaturity']
    sigma = calls.iloc[i]['impliedVolatility']
    
    #calculate greeks using jax formulas and append
    greeks = {'Delta' : jax_delta(S, K, T, r, sigma, q, optType='call'),
              'Gamma' : jax_gamma(S, K, T, r, sigma, q, optType='call'),
              'Theta' : jax_theta(S, K, T, r, sigma, q, optType='call'),
              'Rho' : jax_rho(S, K, T, r, sigma, q, optType='call'),
              'Vega' : jax_vega(S, K, T, r, sigma, q, optType='call')}
    callGreeks = pd.concat([callGreeks, pd.DataFrame([greeks])], ignore_index=True)


display('Our Greeks:', callGreeks)






#existing code

#create empty list to collect values
callGreeks2 = pd.DataFrame(columns=['Delta', 'Gamma', 'Theta', 'Rho', 'Vega'])  

S = stockPrice
r = riskFreeRate
q=dividendYield

for i in range(n):
    K = calls.iloc[i]['strike']
    T = calls.iloc[i]['yearsToMaturity']
    sigma = calls.iloc[i]['impliedVolatility']
    
    #calculate greeks using jax formulas and append
    temp = bs.BlackScholesCall(S=S, K=K, T=T, r=r, sigma=sigma, q=q)
    greeks = {'Delta' : temp.delta(),
              'Gamma' : temp.gamma(),
              'Theta' : temp.theta(),
              'Rho' : temp.rho(),
              'Vega' : temp.vega()}
    callGreeks2 = pd.concat([callGreeks, pd.DataFrame([greeks])], ignore_index=True)

display('Existing Greeks:', callGreeks)

'Our Greeks:'

,Delta,Gamma,Theta,Rho,Vega
0,0.988415,1.6342052e-05,-10.568718,0.07125516,0.22274987
1,0.98719174,3.1068936e-05,-1.2814941,0.16296726,0.32196257
2,0.9859793,4.677557e-05,9.991623,0.2528218,0.41621143
3,0.9852819,6.0533235e-05,8.181709,0.34670645,0.46897417
4,0.98415506,7.831802e-05,16.317055,0.43690854,0.5523414
5,0.9707455,8.800716e-05,598.8176,0.17029195,1.4350173
6,0.9824778,0.0001137633,19.38733,0.6216008,0.6727759
7,0.9816534,0.00013343914,20.316002,0.71400964,0.73058844
8,0.95633745,0.00015833836,859.6012,0.28422517,2.25392
9,0.94844234,0.0002130426,898.7444,0.37953618,2.6660035


'Existing Greeks:'

,Delta,Gamma,Theta,Rho,Vega
0,0.988415,1.6342052e-05,-10.568718,0.07125516,0.22274987
1,0.98719174,3.1068936e-05,-1.2814941,0.16296726,0.32196257
2,0.9859793,4.677557e-05,9.991623,0.2528218,0.41621143
3,0.9852819,6.0533235e-05,8.181709,0.34670645,0.46897417
4,0.98415506,7.831802e-05,16.317055,0.43690854,0.5523414
5,0.9707455,8.800716e-05,598.8176,0.17029195,1.4350173
6,0.9824778,0.0001137633,19.38733,0.6216008,0.6727759
7,0.9816534,0.00013343914,20.316002,0.71400964,0.73058844
8,0.95633745,0.00015833836,859.6012,0.28422517,2.25392
9,0.94844234,0.0002130426,898.7444,0.37953618,2.6660035


Once again we see that we get exactly the same solution as the existing model and this makes sense since we are calculating the same thing. This is useful because now that we understand the JAX library and how powerful it can be, we can use it to calculate other things using the black scholes model.

---

## 4. Implied Volatility using Newton Method
Calculating the valatility implied by the market price for each option

In [146]:
#create loss function that is the difference between theoretical price and actual market option price
#minimize the loss function value

def loss(S, K, T, r, sigmaGuess, price, q=0, optType='call'):
    #price option with sigma guess
    theoreticalPrice = black_scholes(S, K, T, r, sigmaGuess, q, optType)

    #actual price
    marketPrice = price
    
    return abs(theoreticalPrice - marketPrice)


#differentiate loss function with respect to sigmaGuess
#lossGrad is now a function
lossGrad = grad(loss, argnums=4)


### Newton Method

Newton's method is an iterative numerical technique for finding approximations to the roots (or zeros) of a real-valued function.

1. Choose an initial guess \( $x_0$ \).
2. Compute the next approximation using the formula:
   $
   x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
   $
3. Repeat step 2 until \( $|x_{n+1} - x_n|$ \) is sufficiently small (convergence criteria).



### Newton Method for Implied Volatility 

Where $\mathcal{L}$ is the loss function defined above


$$
\sigma_{n+1} = \sigma_n - \frac{\mathcal{L}(\sigma_n)}{\frac{\partial \mathcal{L}}{\partial \sigma}}
$$

Iterative Algorithm: 
1. Make a guess for the volatility
2. Calculate the loss function
3. Check if the loss is less than the tolerance $\epsilon$
- If $|P_{theory}-P_{actual}| < \epsilon$ then stop
- Else, continue
4. Calculate the gradient of the loss function
5. Update valitility guess using the Newton-Raphson formula
6. Repate steps 2-5 until converged or hit the maximum number of iterations

In [147]:
#implement newtons method for IV (default sigma guess is 0.8)

def solve_for_IV(S, K, T, r, price, sigmaGuess = 0.8, q = 0, optType = 'call',
                 N_iterations = 20, epsilon = 0.001, verbose = False):
    #1. Make a guess for the volatility
    sigma = sigmaGuess
    for i in range(N_iterations):
        if verbose:
            print('\nIteration: ', i)

        
    #2. Calculate the loss function
        lossVal = loss(S, K, T, r, sigma, price, q, optType)

        if verbose:
            print("Current Error in Theoretical vs Market Price: ", lossVal)

    #3. Check if the loss is less than the tolerance $\epsilon$
        if abs(lossVal) < epsilon:
            break
        else:
            
    #4. Calculate the gradient of the loss function
            lossGradVal = lossGrad(S, K, T, r, sigma, price, q, optType)
    #5. Update valitility guess using the Newton-Raphson formula
            sigma = sigma - lossVal / lossGradVal

    #6. Repate steps 2-5 until converged or hit the maximum number of iterations
    return sigma

In [177]:
#select an option index to compare the calculated IV to the IV on yahoo finance
optionNum = 42

S = stockPrice
r = riskFreeRate
q = dividendYield
sigmaGuess = 0.5

IVcomp = pd.DataFrame(columns = ['Calculated IV', 'Actual IV'])


K = calls.iloc[optionNum]['strike']
T = calls.iloc[optionNum]['yearsToMaturity']
calculatedIV = solve_for_IV(S, K, T, r, price, sigmaGuess, q)

IVs = {'Calculated IV' : calculatedIV,
        'Actual IV' : calls.iloc[optionNum]['impliedVolatility']}
IVcomp = pd.concat([IVcomp, pd.DataFrame([IVs])], ignore_index=True)
IVcomp

C:\Users\zpetk\AppData\Local\Temp\ipykernel_25204\4137540904.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IVcomp = pd.concat([IVcomp, pd.DataFrame([IVs])], ignore_index=True)


,Calculated IV,Actual IV
0,0.038362943,0.321662


In [ ]:
#select number of options to calculate IV for
n = 10


S = stockPrice
r = riskFreeRate
q = dividendYield
sigmaGuess = 2.0

IVcomp = pd.DataFrame(columns = ['Calculated IV', 'Actual IV'])

for i in range(50,50+n):
    K = calls.iloc[i]['strike']
    T = calls.iloc[i]['yearsToMaturity']
    calculatedIV = solve_for_IV(S, K, T, r, price, sigmaGuess, q)

    IVs = {'Calculated IV' : calculatedIV,
              'Actual IV' : calls.iloc[i]['impliedVolatility']}
    IVcomp = pd.concat([IVcomp, pd.DataFrame([IVs])], ignore_index=True)

IVcomp

C:\Users\zpetk\AppData\Local\Temp\ipykernel_25204\4191933482.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IVcomp = pd.concat([IVcomp, pd.DataFrame([IVs])], ignore_index=True)


,Calculated IV,Actual IV
0,0.23051293,0.345710
1,0.25280705,0.355475
2,0.27468023,0.375006
3,0.29614276,0.404303
4,0.31720373,0.419928
5,0.31700513,0.431646
6,0.33613992,0.457037
7,0.3549504,0.482427
8,0.37344176,0.507817
9,0.39162105,0.507817


The calculated implied volatility (IV) derived from the Black-Scholes model is reasonably close to the actual IV provided by the market. However, it's important to note that this form of the Black-Scholes model is designed for European options, which can only be exercised at expiration. Since the actual options may be American-style, which allows early exercise, the model’s estimates may not fully capture the complexities of such options. Despite these limitations, the model provides a solid approximation, and the discrepancies observed are expected given the assumptions inherent in the Black-Scholes framework.